# CS 121 Lecture 4 : Defining computation

In [ ]:
%%html
<style>

.CodeMirror {
    width: 100vw;
}
S
.container {
    width: 99% !important;
}

.rendered_html {
  font-size:0.8em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}

td {
  font-size: 50px
}
</style>

In [ ]:
def AND(a,b): return a*b

def OR(a,b): return 1 if a+b else 0

def NOT(a): return 1-a

__Q:__ Compute $f:\{0,1\}^3 \rightarrow \{0,1\}$ s.t.
$$f(a,b,c) = \begin{cases}b & a=1 \\ c & a =0 \end{cases}$$
using $AND$, $OR$, and $NOT$

__Solution:__    $f(a,b,c) = (a \wedge b) \vee (\neg a \wedge c)$

In [ ]:
# (a ∧ b) ∨ (¬a ∧ c)
C = circuit(lambda a,b,c: OR( AND(a,b) , AND( NOT(a),c ) ) )
C

In [ ]:
C(0,0,1, render = True)

In [ ]:
table(C)

In [ ]:
print(circuit2prog(C))
C.draw()

__Def I (3.5):__ <font color="red">_Boolean Circuit_</font> $C$ is a DAG where vertices  
are <font color="blue">_inputs_</font> labeled   `X[0]` ... `X[n-1]` or <font color="blue">_gates_</font>
labeled  $\wedge$/$\vee$/$\neg$.
<font color="blue">_Output gates_</font> are additionally labeled with  `Y[0]` ... `Y[m-1]`.<br>
For $x\in \{0,1\}^n$,  we denote the output of $C$ on input $x$ by $C(x)$.

__Def II (3.7):__ <font color="red">_AON-CIRC program_</font> $P$ has lines of form
`foo = AND(bar,blah)` , `foo = OR(bar,blah)` or `foo = NOT(blah)`. 
<font color="blue">_Input variables_</font> are named  `X[0]`,...,`X[n-1]`and 
<font color="blue">_output variables_</font> are named  `Y[0]`..`Y[m-1]`. <br>
For $x\in \{0,1\}^n$, we denote the output of $P$ on input $x$ by $P(x)$. 

__Theorem III (3.8):__ For every $f:\{0,1\}^n \rightarrow \{0,1\}^m$ and $s\in \mathbb{N}$, 
$f$ is computable by a Boolean circuit of $s$ gates if and only if 
$f$ is computable by an AON-CIRC program of $s$ lines.

In [ ]:
def EVAL(code,X):
    """Evaluate code on input X."""
    n,m = numinout(code) # helper function - get number of inputs and outputs
    
    vtable = { f"X[{i}]":int(X[i]) for i in range(n)}
    
    for line in code.split("\n"):
        if not line: continue
        foo,op,bar,blah = parseline(line,2) 
        # helper function - split "foo = OP(,blah)" to list ["foo","OP","bar","blah"]
        # 2 is num of arguments to expect: blah is empty if it's missing
        if op=="NOT": vtable[foo] = NOT(vtable[bar])
        if op=="AND": vtable[foo] = AND(vtable[bar],vtable[blah])
        if op=="OR": vtable[foo] =  OR(vtable[bar],vtable[blah])
    
    return [vtable[f"Y[{j}]"] for j in range(m)]            

__Q:__ Write an AON-CIRC program that computes $XOR:\{0,1\}^2 \rightarrow \{0,1\}$ where $XOR(a,b) = a+b \mod 2$

code = r'''
foo = AND(X[0],X[1])
bar = NOT(foo)
or  = OR(X[0],X[1])
Y[0] = OR(foo,OR)'''[1:]

In [ ]:
xorcode = r"""
t1      = AND(X[0],X[1])
nott1   = NOT(t1)
t2      = OR(X[0],X[1])
Y[0]    = AND(nott1,t2)
"""[1:]
table(xorcode)

In [ ]:
circuit(xorcode)

Back to powerpoint

....

__Q:__ Write NAND-CIRC code to compute $XOR:\{0,1\}^2 \rightarrow \{0,1\}$ such that $XOR(a,b) = a+b \mod 2$.

Reminder AON-CIRC code is:
```python
t1      = AND(X[0],X[1])
nott1   = NOT(t1)
t2      = OR(X[0],X[1])
Y[0]    = AND(nott1,t2)
```

In [ ]:
def parseline(line, numargs = 2):
    """Parse a line of the form foo = 'OP( bar1, bar2, bar3)'
       to ['foo','OP','bar1','bar2','bar2'].
       If numargs > number of arguments in line then add empty strings to the list as needed."""
    if not line: return [""]*(2+numargs)
    i = line.find("=")
    j = line.find("(")
    k = line.find(")")
    if i<0 or j<0 or k<0: raise Exception(f"Line not formatted properly: {line}")
    args = [a.strip() for a in line[j+1:k].split(",")]
    if len(args)<numargs: args += [""]*(numargs-len(args))
    return [line[:i].strip() , line[i+1:j].strip()] + args

In [ ]:
def AON2NAND(code):
    output = ""; counter = 0
    for line in code.split("\n"):
        foo,op,bar,blah = parseline(line) 
        # helper: parses "foo = op(bar,blah)"
        if op=="NOT":
            output += f"{foo} = NAND({bar},{bar})\n"
        if op=="AND": 
            output += f"temp_{counter} = NAND({bar},{blah})\n"
            output += f"{foo} = NAND(temp_{counter},temp_{counter})\n"
            counter +=1
        if op=="OR":
            output += f"temp_{counter} = NAND({bar},{bar})\n"
            output += f"temp_{counter+1} = NAND({blah},{blah})\n"
            output += f"{foo} = NAND(temp_{counter},temp_{counter+1})\n"
            counter +=2
    return output

In [ ]:
xornand = AON2NAND(xorcode)
print(xornand)

In [ ]:
circuit(xornand).draw()
table(xornand)

In [ ]:
# Q: what does this program compute?
mystery='''
u = NAND(X[0],X[1])
v = NAND(X[0],u)
w = NAND(X[1],u) 
Y[0] = NAND(v,w)'''[1:]

In [ ]:
table(mystery)

In [ ]:
circuit(mystery)

__Q:__ Design a NAND circuit to compute $XOR4:\{0,1\}^4 \rightarrow \{0,1\}$ where $XOR4(a,b,c,d) = a+b+c+d \mod 2$

In [ ]:
def XOR2(a,b):
    u = NAND(a,b)
    v = NAND(a,u)
    w = NAND(b,u)
    return NAND(v,w)

def XOR4(a,b,c,d):
    u = XOR2(a,b)
    v = XOR2(c,d)
    return XOR2(u,v)

XOR4circ = circuit(XOR4)

In [ ]:
XOR4circ

In [ ]:
table(XOR4circ)

In [ ]:
print(circuit2prog(XOR4circ))

## Next Up:

__Theorem:__ For _every_ function $f:\{0,1\}^n \rightarrow \{0,1\}^m$ there exists a Boolean circuit to compute $f$.

__Corollary:__ For every such $f$, there is AON-CIRC program/NAND circuit/NAND-CIRC program to compute $f$.

![](pizzaparty.png)